In [1]:
# Implementation of Newton's method, Steepest Descent (constant step size, & Armijo's rule), and Momentum Descent
# This version: for HW5
using Printf
using LinearAlgebra
using Plots

[ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]

Please submit a bug report with steps to reproduce this fault, and any error messages that follow (in their entirety). Thanks.
Exception: EXCEPTION_ACCESS_VIOLATION at 0x7b4e8b47 -- jl_svecref at C:/workdir/src\julia.h:989 [inlined]
jl_array_grow_at_end at C:/workdir/src\array.c:880
in expression starting at none:1
jl_svecref at C:/workdir/src\julia.h:989 [inlined]
jl_array_grow_at_end at C:/workdir/src\array.c:880
ijl_array_grow_end at C:/workdir/src\array.c:955
_growend! at .\array.jl:1014 [inlined]
push! at .\array.jl:1061 [inlined]
load_libs at C:\Users\shimk\.julia\packages\GR\yBe3g\src\funcptrs.jl:54
get_func_ptr at C:\Users\shimk\.julia\packages\GR\yBe3g\src\funcptrs.jl:75
get_func_ptr at C:\Users\shimk\.julia\packages\GR\yBe3g\src\funcptrs.jl:75 [inlined]
libGR_ptr at C:\Users\shimk\.julia\packages\GR\yBe3g\src\funcptrs.jl:84 [inlined]
setcharheight at C:\Users\shimk\.julia\packages\GR\yBe3g\src\GR.jl:1552
unkno

LoadError: Failed to precompile Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80] to "C:\\Users\\shimk\\.julia\\compiled\\v1.9\\Plots\\jl_93A8.tmp".

In [2]:
# load data
include("data-hw5.jl");
z = data[:,1];
y = data[:,2];
n = length(z);   # number of data points

In [3]:
# set parameters here, for all optimization algorithms
tol = 1e-6;     # tolerance on norm of gradient
MaxIter = 1e5;  # maximum number of iterations of gradient descent
MaxIterNewton = 500;  # maximum number of iterations for Newton-type methods. 

### Defining Functions

In [4]:
# sigmoidal function, and its derivatives
function s(x)
   return 1/(1+exp(-x));
end

function sp(x)
    return exp(-x)/(1+exp(-x))^2;
end

function spp(x)
    return 2*exp(-2*x)/(1+exp(-x))^3 - exp(-x)/(1+exp(-x))^2;
end

# model function: y = m(z,x), where x are parameters
function m(z,x)
    a = x[1];
    b = x[2];
    return s(a*z+b);
end

# gradient of model function
function Dm(z,x)
   a = x[1]; b = x[2]; 
   g1 = z*sp(a*z+b);
   g2 = sp(a*z + b); # Inserted
   return [g1;g2];
end

# Hessian of model function
function Dm2(z,x)
   a = x[1]; b = x[2]; 
   H = zeros(2,2);
   ### YOU INSERT ###
   H[1,1] = z*z*spp(a*z + b);
   H[1,2] = z*spp(a*z+b);
   H[2,1] = z*spp(a*z+b);
   H[2,2] = spp(a*z+b);
   ### END INSERT ###
   return H;
end

Dm2 (generic function with 1 method)

In [38]:
# Loss function
function F(x)
   L = 0;
   for i=1:n
      L = L + 0.5*(y[i] - m(z[i],x))^2;
   end
   return L;
end

# gradient of Loss function
function DF(x)
   g = zeros(2);
   for i=1:n
      g = g-(y[i]-m(z[i],x))*Dm(z[i],x);
   end
   return g;
end

# Hessian of Loss function
function DF2(x)
    H = zeros(2,2);
    for i=1:n
      H = H - ( (y[i]-m(z[i],x))*Dm2(z[i],x) - Dm(z[i],x)*Dm(z[i],x)');
   end
   return H;
end

DF2 (generic function with 1 method)

In [13]:
# steepest descent algorithm, with constant step size
# input: 
#    x0 = initial point
#    alpha = step size. Constant, in this algorithm.

# output: 
#    xsave = list of points

function SteepestDescent(x0,alpha)

   # setup for steepest descent
   x = x0;
   successflag = false;
   xsave = zeros(length(x0),MaxIter+1);
   xsave[:,1] = x0;

   # perform steepest descent iterations
   for iter = 1:MaxIter

       # compute gradient
       Fgrad = DF(x);

       # print info
       # @printf("x = %11.10f, %11.10f, F(x) = %10.8f, |grad F| = %10.8f \n", x[1],x[2],F(x),sqrt(Fgrad'*Fgrad));

       # check if gradient is small enough
       if sqrt(Fgrad'*Fgrad) < tol
          @printf("x = %11.10f, %11.10f, F(x) = %10.8f, |grad F| = %10.8f \n", x[1],x[2],F(x),sqrt(Fgrad'*Fgrad));
            @printf("Converged after %d iterations, function value %f\n", iter, F(x))
          successflag = true;
          xsave = xsave[:,1:iter];
          break;
       end

       # perform steepest descent step
       x = x - alpha*Fgrad;
       
       # save point
       xsave[:,iter+1] = x;
   end
   if successflag == false
       @printf("Failed to converge after %d iterations, function value %f\n", MaxIter, F(x));
   end
   return xsave;
end

SteepestDescent (generic function with 1 method)

In [24]:
# steepest descent algorithm, with Armijo's rule for backtracking
# input: 
#    x0 = initial point
#    c1 = slope, in Armijo's rule.
# output: 
#    xsave = list of points
#
function SteepestDescentArmijo(x0)

   # parameters
   alpha0 = 10.0;    # initial value of alpha, to try in backtracking
   eta = 0.5;       # factor with which to scale alpha, each time you backtrack
   MaxBacktrack = 20;  # maximum number of backtracking steps
   c1 = 1e-2;       # slope factor, in Armijo's rule

   # setup for steepest descent
   x = x0;
   successflag = false;   
   xsave = zeros(length(x0),MaxIter);
   xsave[:,1] = x0;

   # perform steepest descent iterations
   for iter = 1:MaxIter

      alpha = alpha0;

      # compute gradient
       Fgrad = DF(x);

       # print info
       # @printf("x = %11.10f, %11.10f, F(x) = %10.8f, |grad F| = %10.8f \n", x[1],x[2],F(x),sqrt(Fgrad'*Fgrad));

      # check if norm of gradient is small enough
      if sqrt(Fgrad'*Fgrad) < tol
        @printf("x = %11.10f, %11.10f, F(x) = %10.8f, |grad F| = %10.8f \n", x[1],x[2],F(x),sqrt(Fgrad'*Fgrad));
         @printf("Converged after %d iterations, function value %f\n", iter, F(x))
         successflag = true;
         xsave = xsave[:,1:iter];
         break;
      end

      # perform line search
      Fval = F(x);
      for k = 1:MaxBacktrack
         x_try = x - alpha*Fgrad;
         Fval_try = F(x_try);
         if (Fval_try > Fval - c1*alpha *Fgrad'Fgrad)
            alpha = alpha * eta;
         else
            Fval = Fval_try;
            x = x_try;
            break;
         end
      end

      # save point
      xsave[:,iter+1] = x;

      # print how we're doing, every 10 iterations
      #if (iter%5==0)
      #   @printf("iter: %d: alpha: %f, %f, %f, %f\n", iter, alpha, x[1], x[2], F(x))
      #end 
   end

   if successflag == false
       @printf("Failed to converge after %d iterations, function value %f\n", MaxIter, F(x))
   end

   return xsave
end

SteepestDescentArmijo (generic function with 1 method)

In [165]:
# Newton's algorithm
# input: 
#    x0 = initial point, a 2-vector (e.g. x0=[1;2])
# output: 
#    xsave = list of points

function Newtonv0(x0) # Original Version

   # setup 
   x = x0;
   successflag = false;
   xsave = zeros(length(x0),MaxIter+1);
   xsave[:,1] = x0;

   # iterate
   for iter = 1:MaxIterNewton
      
       # compute gradient
       Fgrad = DF(x);

       #@printf("x = %11.10f, %11.10f, F(x) = %10.8f, |grad F| = %10.8f \n", x[1],x[2],F(x),sqrt(Fgrad'*Fgrad));

       # check whether gradient is small enough
       if sqrt(Fgrad'*Fgrad) < tol
            @printf("x = %11.10f, %11.10f, F(x) = %10.8f, |grad F| = %10.8f \n", x[1],x[2],F(x),sqrt(Fgrad'*Fgrad));
          @printf("\nConverged after %d iterations, F(x) = %f\n", iter, F(x));
          println("x = ", x');
          successflag = true;
          xsave = xsave[:,1:iter];
          break;
       end

       # compute Hessian
       H = DF2(x);

       # check if hessian is positive definite
       ### YOU INSERT ###
        

       # Newton step
       x = x - inv(H) * Fgrad;    # normally you don't actually compute a matrix inverse
       
       # save point
       xsave[:,iter+1] = x;
   end
    
   if successflag == false
       @printf("Failed to converge after %d iterations, function value %F\n", MaxIter, F(x))
   end
    
   return xsave;
end

Newtonv0 (generic function with 1 method)

In [199]:
# Newton's algorithm
# input: 
#    x0 = initial point, a 2-vector (e.g. x0=[1;2])
# output: 
#    xsave = list of points

function Newtonv1(x0)
    # setup 
    x = x0
    successflag = false
    xsave = zeros(length(x0), MaxIter + 1)
    xsave[:, 1] = x0

    # iterate
    for iter = 1:MaxIterNewton
        try
            # compute gradient
            Fgrad = DF(x)

            #@printf("x = %11.10f, %11.10f, F(x) = %10.8f, |grad F| = %10.8f \n", x[1],x[2],F(x),sqrt(Fgrad'*Fgrad));

            # check whether gradient is small enough
            if sqrt(Fgrad' * Fgrad) < tol
                @printf("\nConverged after %d iterations, F(x) = %f\n", iter, F(x))
                println("x = ", x')
                successflag = true
                xsave = xsave[:, 1:iter]
                break
            end

            # compute Hessian
            H = DF2(x)

            # Newton step
            x = x - inv(H) * Fgrad  # normally you don't actually compute a matrix inverse

            # save point
            xsave[:, iter + 1] = x
        catch e
            println("Caught an error: ", e)
            println("Skipping iteration ", iter)
            @printf("Failed to converge after %d iterations, function value %F\n", iter, F(x))
            break  # Break out of the loop if an error occurs
            
        end
    end

    if successflag == false
        @printf("Failed to converge after %d iterations, function value %F\n", MaxIter, F(x))
    end

    return xsave
end


Newtonv1 (generic function with 1 method)

In [198]:
# Newton's algorithm
# input: 
#    x0 = initial point, a 2-vector (e.g. x0=[1;2])
# output: 
#    xsave = list of points

function Newtonv2(x0)
    # setup 
    x = x0
    successflag = false
    xsave = zeros(length(x0), MaxIter + 1)
    xsave[:, 1] = x0

    # iterate
    for iter = 1:MaxIterNewton
        try
            # compute gradient
            Fgrad = DF(x)

            #@printf("x = %11.10f, %11.10f, F(x) = %10.8f, |grad F| = %10.8f \n", x[1],x[2],F(x),sqrt(Fgrad'*Fgrad));

            # check whether gradient is small enough
            if sqrt(Fgrad' * Fgrad) < tol
                @printf("\nConverged after %d iterations, F(x) = %f\n", iter, F(x))
                println("x = ", x')
                successflag = true
                xsave = xsave[:, 1:iter]
                break
            end

            # compute Hessian
            H = DF2(x)

            # check if hessian is positive definite
            min_eval = findmin(eigen(H).values)[1]
            print("\n Minimum eigenvalue: ", min_eval, "\n")

            # Newton step
            x = x - inv(H) * Fgrad  # normally you don't actually compute a matrix inverse

            # save point
            xsave[:, iter + 1] = x
        catch e
            println("Caught an error: ", e)
            println("Skipping iteration ", iter)
            @printf("Failed to converge after %d iterations, function value %F\n", iter, F(x))
            break  # Break out of the loop if an error occurs
            
        end
    end

    if successflag == false
        @printf("Failed to converge after %d iterations, function value %F\n", MaxIter, F(x))
    end

    return xsave
end


Newtonv2 (generic function with 1 method)

In [295]:
# Newton's algorithm, with Hessian modification
# input: 
#    x0 = initial point, a 2-vector (e.g. x0=[1;2])
# output: 
#    xsave = list of points
#
function NewtonModified(x0)

   # parameters
   backtrack = false;  # whether or not to apply backtracking
   eta = 0.5;       # factor with which to scale alpha, each time you backtrack
   MaxBacktrack = 20;  # maximum number of backtracking steps
   c1 = 1e-3;       # slope factor, in Armijo's rule


   # setup 
   x = x0;
   successflag = false;
   xsave = zeros(length(x0),MaxIter+1);
   xsave[:,1] = x0;

   # iterate
   for iter = 1:MaxIterNewton
        try
            # compute gradient
           Fgrad = DF(x);
            

           # check whether gradient is small enough
           if sqrt(Fgrad'*Fgrad) < tol
              @printf("\nConverged after %d iterations, F(x) = %f\n", iter, F(x));
              println("\n x = ", x');
              successflag = true;
              xsave = xsave[:,1:iter];
              break;
           end

           # Find multiple of identity to add to Hessian, to make it positive definite
           H = DF2(x);   # Hessian
            tau = 0;
            delta = 2;

           ### YOU INSERT ### 
            B = H + tau*I(size(H)[1]);    # YOU MUST POSSIBLY MODIFY B HERE

            min_B_eval = findmin(eigen(B).values)[1]
            if min_B_eval < delta
                tau = max(tau, (delta - min_B_eval) + 0.01)
                # print("\n min_B_eval:", min_B_eval, ", tau: ", tau, "\n")
            end

            ### END INSERT ### 

           d = inv(B)*Fgrad;  # descent direction
           alpha = 1;

           # find step size alpha, using backtracking
           if backtrack
              Fval = F(x);
              for k = 1:MaxBacktrack
                x_try = x - alpha*d;
                Fval_try = F(x_try);
                if (Fval_try > Fval - c1*alpha *Fgrad'*d)
                   alpha = alpha * eta;
                else
                   break;
                end
             end
          end

          @printf("x = %11.10f, %11.10f, F(x) = %10.8f, |grad F| = %10.8f, tau = %6.4f, alpha = %6.4f \n", x[1],x[2],F(x),sqrt(Fgrad'*Fgrad),tau, alpha);
           # take step
           x = x - alpha*d;    # normally you don't actually compute a matrix inverse

           # save point
           xsave[:,iter+1] = x;
       
        catch e
            println("Caught an error: ", e)
            println("Skipping iteration ", iter)
            @printf("Failed to converge after %d iterations, function value %F\n", iter, F(x))
            break  # Break out of the loop if an error occurs
        end
   end
   if successflag == false
       @printf("Failed to converge after %d iterations, function value %F\n", MaxIter, F(x))
   end
   return xsave;
end


NewtonModified (generic function with 1 method)

In [311]:
# Newton's algorithm, with Hessian modification
# input: 
#    x0 = initial point, a 2-vector (e.g. x0=[1;2])
# output: 
#    xsave = list of points
#
function NewtonModified_Backtrack(x0)

   # parameters
   backtrack = true;  # whether or not to apply backtracking
   eta = 0.5;       # factor with which to scale alpha, each time you backtrack
   MaxBacktrack = 20;  # maximum number of backtracking steps
   c1 = 1e-3;       # slope factor, in Armijo's rule


   # setup 
   x = x0;
   successflag = false;
   xsave = zeros(length(x0),MaxIter+1);
   xsave[:,1] = x0;

   # iterate
   for iter = 1:MaxIterNewton
        try
            # compute gradient
           Fgrad = DF(x);
            

           # check whether gradient is small enough
           if sqrt(Fgrad'*Fgrad) < tol
              @printf("\nConverged after %d iterations, F(x) = %f\n", iter, F(x));
              println("\n x = ", x');
              successflag = true;
              xsave = xsave[:,1:iter];
              break;
           end

           # Find multiple of identity to add to Hessian, to make it positive definite
           H = DF2(x);   # Hessian
            tau = 0;
            delta = 2;

           ### YOU INSERT ### 
            B = H + tau*I(size(H)[1]);    # YOU MUST POSSIBLY MODIFY B HERE

            min_B_eval = findmin(eigen(B).values)[1]
            if min_B_eval < delta
                tau = max(tau, (delta - min_B_eval) + 0.01)
                # print("\n min_B_eval:", min_B_eval, ", tau: ", tau, "\n")
            end

            ### END INSERT ### 

           d = inv(B)*Fgrad;  # descent direction
           alpha = 1;

           # find step size alpha, using backtracking
           if backtrack
              Fval = F(x);
              for k = 1:MaxBacktrack
                x_try = x - alpha*d;
                Fval_try = F(x_try);
                if (Fval_try > Fval - c1*alpha *Fgrad'*d)
                   alpha = alpha * eta;
                else
                   break;
                end
             end
          end

          # @printf("x = %11.10f, %11.10f, F(x) = %10.8f, |grad F| = %10.8f, tau = %6.4f, alpha = %6.4f \n", x[1],x[2],F(x),sqrt(Fgrad'*Fgrad),tau, alpha);
           # take step
           x = x - alpha*d;    # normally you don't actually compute a matrix inverse

           # save point
           xsave[:,iter+1] = x;
       
        catch e
            println("Caught an error: ", e)
            println("Skipping iteration ", iter)
            @printf("Failed to converge after %d iterations, function value %F\n", iter, F(x))
            break  # Break out of the loop if an error occurs
        end
   end
   if successflag == false
       @printf("Failed to converge after %d iterations, function value %F\n", MaxIter, F(x))
   end
   return xsave;
end


NewtonModified_Backtrack (generic function with 1 method)

In [10]:
# plot contours of function
function plotcontours(xl,yl)
   xmin = -xl;
   xmax = xl;
   ymin = -yl;
   ymax = yl;
   x = xmin:0.05:xmax;
   y = ymin:0.05:ymax;
   Z = zeros(length(y), length(x));
   for i=1:length(x)
      for j=1:length(y)
        Z[j,i] = F([x[i]; y[j]]);
      end
   end
   
   contourf(x,y,Z,levels=20,aspect_ratio=:equal,fill=(true,cgrad(:inferno,rev=true)));
   #contourf(x,y,Z,levels=30,fill=(true,cgrad(:haline ,[0,0.1,1.0])));
   #contourf(x,y,Z,levels=30,fill=(true,cgrad(:deep,[0,0.2,1.0])));
   #contourf(x,y,Z,levels=30,fill=(true,cgrad(:inferno,scale=:exp)));

   # color schemes, see:
   #   http://docs.juliaplots.org/latest/generated/colorschemes/#cmocean 
   # see also
   #   https://github.com/JuliaPlots/ExamplePlots.jl/blob/master/notebooks/cgrad.ipynb
   #   https://docs.juliaplots.org/latest/generated/colorschemes/
   #   https://github.com/JuliaGraphics/Colors.jl/blob/master/src/names_data.jl
end

plotcontours (generic function with 1 method)

In [11]:
# plot data on top of contour plot
function plotdata(xsave,str)
   K= length(xsave[1,:]);
   println("k = ", K)
   plot!(xsave[1,1:1:K], xsave[2,1:1:K], lw = 2, marker=2, label = str,legend=:bottomright);
   #savefig("plot.png")
end

plotdata (generic function with 1 method)

### Code Implementation for Questions

In [27]:
# Q5A - testing Dm(z, x), Dm^2(z,x)
Dm(2, [1,0])
Dm2(2, [1,0])

2×2 Matrix{Float64}:
 -0.31985   -0.159925
 -0.159925  -0.0799625

In [22]:
# Q5B - Steepest Descent w/ Constant Step Size
tol = 1e-6;     # tolerance on norm of gradient
MaxIter = 100000;
alpha_lvl = [0.05, 0.1, 0.3]

for i in range(1,5)
    x0 = randn(2)
    for alpha in alpha_lvl
        print("iter: ",i ," alpha: ", alpha, " starting points:", x0, "\n")
        SteepestDescent(x0, alpha)
        print("\n")
    end
end

iter: 1 alpha: 0.05 starting points:[-0.5021926059012495, -0.5342527158461803]
x = 0.7011309324, -3.4622019552, F(x) = 0.31707947, |grad F| = 0.00000100 
Converged after 5591 iterations, function value 0.317079

iter: 1 alpha: 0.1 starting points:[-0.5021926059012495, -0.5342527158461803]
x = 0.7011309403, -3.4622019964, F(x) = 0.31707947, |grad F| = 0.00000100 
Converged after 2778 iterations, function value 0.317079

iter: 1 alpha: 0.3 starting points:[-0.5021926059012495, -0.5342527158461803]
Failed to converge after 100000 iterations, function value 0.384422

iter: 2 alpha: 0.05 starting points:[0.10848592319440979, -2.4168982856745216]
x = 0.7011309377, -3.4622019832, F(x) = 0.31707947, |grad F| = 0.00000100 
Converged after 5063 iterations, function value 0.317079

iter: 2 alpha: 0.1 starting points:[0.10848592319440979, -2.4168982856745216]
x = 0.7011411334, -3.4622551706, F(x) = 0.31707947, |grad F| = 0.00000100 
Converged after 3362 iterations, function value 0.317079

iter: 2

In [201]:
# 5B - Steepest Descent w/ Armijo
tol = 1e-6;     # tolerance on norm of gradient
MaxIter = 100000;
alpha_lvl = [0.05, 0.1, 0.3]

for i in range(1,5)
    x0 = randn(2)
    print("iter: ", i, " starting points: ", x0, "\n")
    SteepestDescentArmijo(x0)
    print("\n")
end

iter: 1 starting points: [0.1418047773644099, -0.16141338682428366]
x = 0.7011329956, -3.4622130507, F(x) = 0.31707947, |grad F| = 0.00000098 
Converged after 1487 iterations, function value 0.317079

iter: 2 starting points: [1.0642505462225191, 1.4122750614559334]
x = 0.7011392273, -3.4622455472, F(x) = 0.31707947, |grad F| = 0.00000099 
Converged after 2151 iterations, function value 0.317079

iter: 3 starting points: [-0.2985051515380857, -1.9002592599443566]
x = 65.2681485795, -3.1899207188, F(x) = 3.70013409, |grad F| = 0.00000099 
Converged after 30938 iterations, function value 3.700134

iter: 4 starting points: [1.65756724301468, 0.03596665548738266]
x = 0.7011331060, -3.4622129617, F(x) = 0.31707947, |grad F| = 0.00000099 
Converged after 1337 iterations, function value 0.317079

iter: 5 starting points: [0.035231997229772785, -0.541746713303703]
x = 0.7011326591, -3.4622112815, F(x) = 0.31707947, |grad F| = 0.00000100 
Converged after 1296 iterations, function value 0.317079

In [203]:
# 5C - Newton's Method
tol = 1e-6;     # tolerance on norm of gradient
MaxIterNewton = 1000;

for i in range(1,5)
    x0 = randn(2)
    print("iter: ", i, " , starting points: ", x0, "\n")
    Newtonv1(x0)
    print("\n")
end

iter: 1 , starting points: [0.43851347604982266, -0.20602953315044753]

Converged after 15 iterations, F(x) = 3.700133
x = [34.61213811697523 -3.1892367883780666]

iter: 2 , starting points: [-0.22691254145630663, -0.09630099388813051]
Caught an error: ArgumentError("matrix contains Infs or NaNs")
Skipping iteration 18
Failed to converge after 18 iterations, function value 26.248986
Failed to converge after 100000 iterations, function value 26.248986

iter: 3 , starting points: [1.315219835985126, -0.4244819859753539]
Caught an error: ArgumentError("matrix contains Infs or NaNs")
Skipping iteration 6
Failed to converge after 6 iterations, function value 25.115568
Failed to converge after 100000 iterations, function value 25.115568

iter: 4 , starting points: [0.3749059125175739, -2.1177100043706467]

Converged after 7 iterations, F(x) = 0.317079
x = [0.7011360304332431 -3.4622285543058413]

iter: 5 , starting points: [0.24238154004425347, 0.8144804389232957]

Converged after 17 iterati

In [204]:
# 5E - Newton's Method
tol = 1e-6;     # tolerance on norm of gradient
MaxIterNewton = 500;

for i in range(1,5)
    x0 = randn(2)
    print("iter: ", i, " , starting points: ", x0, "\n")
    Newtonv2(x0)
    print("\n")
end

iter: 1 , starting points: [-1.418900641788563, -0.5311860376083344]

 Minimum eigenvalue: -1.2898189461900165

 Minimum eigenvalue: -0.3268018258116341

 Minimum eigenvalue: -0.09504796802586057

 Minimum eigenvalue: -0.06635112937675093

 Minimum eigenvalue: -0.02161228924128705

 Minimum eigenvalue: -0.003034336064654262
Caught an error: ArgumentError("matrix contains Infs or NaNs")
Skipping iteration 7
Failed to converge after 7 iterations, function value 22.232779
Failed to converge after 100000 iterations, function value 22.232779

iter: 2 , starting points: [1.0577820821166213, -1.0254683613218465]

 Minimum eigenvalue: -2.168112523170202

 Minimum eigenvalue: -0.6819522100212643

 Minimum eigenvalue: -0.22665142537604183

 Minimum eigenvalue: -0.004885564457333229

 Minimum eigenvalue: -0.0015655816631194477

 Minimum eigenvalue: -0.0004106545587024223

 Minimum eigenvalue: -0.0001063344163991526

 Minimum eigenvalue: -3.100200369507162e-5

 Minimum eigenvalue: -1.0331935517344

In [301]:
# 5F - Modified Newton
tol = 1e-6;     # tolerance on norm of gradient
MaxIterNewton = 500;

for i in range(1,10)
    x0 = randn(2)
    print("iter: ", i, " , starting points: ", x0, "\n")
    NewtonModified(x0)
    print("\n")
end

iter: 1 , starting points: [2.124054980511592, 0.2940127394627716]
x = 2.1240549805, 0.2940127395, F(x) = 3.60462024, |grad F| = 0.42642658, tau = 2.2678, alpha = 1.0000 
x = 4.2237177453, -0.4086569006, F(x) = 3.61398912, |grad F| = 0.21584268, tau = 2.0725, alpha = 1.0000 
x = 8.2875211997, -1.8158399297, F(x) = 3.62590276, |grad F| = 0.08778250, tau = 2.0566, alpha = 1.0000 
x = 12.1326627071, -2.4690790052, F(x) = 3.67657905, |grad F| = 0.02854589, tau = 2.0311, alpha = 1.0000 
x = 15.0589134116, -2.8496618235, F(x) = 3.69180514, |grad F| = 0.01000950, tau = 2.0176, alpha = 1.0000 
x = 17.5827551673, -3.0836466585, F(x) = 3.69709590, |grad F| = 0.00354740, tau = 2.0124, alpha = 1.0000 
x = 19.7877416579, -3.1761237932, F(x) = 3.69902094, |grad F| = 0.00125944, tau = 2.0106, alpha = 1.0000 
x = 21.8208804737, -3.1890384465, F(x) = 3.69972556, |grad F| = 0.00045662, tau = 2.0101, alpha = 1.0000 
x = 23.8239642029, -3.1892409007, F(x) = 3.69998394, |grad F| = 0.00016780, tau = 2.0100,


 x = [-69.19477065531922 1090.3641109011564]

iter: 8 , starting points: [0.7227656263280802, 0.5353668201782649]
x = 0.7227656263, 0.5353668202, F(x) = 3.35704300, |grad F| = 1.30043216, tau = 3.8375, alpha = 1.0000 
x = 1.5331691317, 0.5072290913, F(x) = 3.60759338, |grad F| = 0.59345391, tau = 2.5059, alpha = 1.0000 
x = 3.0079908814, 0.1232874089, F(x) = 3.64662386, |grad F| = 0.29967374, tau = 2.1284, alpha = 1.0000 
x = 6.1414235249, -0.8981965365, F(x) = 3.63727170, |grad F| = 0.13652411, tau = 2.0467, alpha = 1.0000 
x = 10.2689583527, -1.8855800689, F(x) = 3.67040266, |grad F| = 0.04543767, tau = 2.0316, alpha = 1.0000 
x = 13.5349590315, -2.4111120229, F(x) = 3.68943422, |grad F| = 0.01567881, tau = 2.0181, alpha = 1.0000 
x = 16.3842324463, -2.7984008017, F(x) = 3.69612902, |grad F| = 0.00550193, tau = 2.0127, alpha = 1.0000 
x = 18.9226546098, -3.0541561091, F(x) = 3.69863123, |grad F| = 0.00190542, tau = 2.0108, alpha = 1.0000 
x = 21.1605341951, -3.1681612385, F(x) = 3.6

In [323]:
# 5G - Modified Newton with BackTracking
tol = 1e-6;     # tolerance on norm of gradient
MaxIterNewton = 500;

for i in range(1,10)
    x0 = randn(2)
    print("iter: ", i, " , starting points: ", x0, "\n")
    NewtonModified_Backtrack(x0)
    print("\n")
end

iter: 1 , starting points: [0.2911877849718705, 0.3984132240326721]
Failed to converge after 100000 iterations, function value 2.861459

iter: 2 , starting points: [1.4942031797891338, -2.203932541074327]
Failed to converge after 100000 iterations, function value 2.112521

iter: 3 , starting points: [-3.1166446400927725, -0.8890603580657512]
Failed to converge after 100000 iterations, function value 22.631540

iter: 4 , starting points: [-0.5201770617612161, 0.6760163673003925]
Failed to converge after 100000 iterations, function value 21.857231

iter: 5 , starting points: [0.2584889160366705, -0.8769875010146798]

Converged after 8 iterations, F(x) = 0.317079

 x = [0.701135994405086 -3.4622283845968242]

iter: 6 , starting points: [1.2972833409154232, 0.07386455113262047]
Failed to converge after 100000 iterations, function value 3.269707

iter: 7 , starting points: [0.05173491794923106, -0.9817526115437116]

Converged after 10 iterations, F(x) = 0.317079

 x = [0.7011360376930422 -3